In [73]:
import pandas as pd

df = pd.read_csv("joined.csv")
df = df.sort_values(by=['platformgameid', 'eventtime'])

display(df)

,gametime_inhib,platformgameid,killerteamid_inhib,gametime_tower,killerteamid_tower,gametime_kill,killerteamid_kill,eventtime,blue_teamid,esportsgameid,red_teamid
0,1708748,ESPORTSTMNT01:1110148,100,939441,100,245612,100,2019-05-01T09:11:39.980Z,1.007832e+17,101978171958054800,9.956641e+16
1,1602302,ESPORTSTMNT01:1110162,100,677086,100,192537,100,2019-05-01T10:15:29.004Z,9.876799e+16,101978171963953057,9.956641e+16
2,1546286,ESPORTSTMNT01:1110177,100,836608,100,207700,100,2019-05-01T11:15:13.609Z,1.007832e+17,101978171979157453,9.956641e+16
3,1628728,ESPORTSTMNT01:1110187,100,626524,200,359245,200,2019-05-01T12:09:47.993Z,9.876799e+16,101978171965722534,9.956641e+16
4,1694081,ESPORTSTMNT01:1110198,100,895424,100,148483,100,2019-05-01T13:01:30.206Z,9.956641e+16,101978171973586877,9.956641e+16
...,...,...,...,...,...,...,...,...,...,...,...
25182,1700113,LPL_B:330027,100,885600,100,294277,100,2023-07-17T09:14:18.420Z,9.876799e+16,110428848767940944,9.956640e+16
25183,1658796,LPL_B:330032,200,687897,200,227140,100,2023-07-17T10:03:31.166Z,9.956640e+16,110428848767940945,9.876799e+16
25184,1946801,LPL_B:330033,200,900364,200,505054,100,2023-07-17T11:17:17.592Z,9.956640e+16,110428848767875248,9.876799e+16
25185,1447915,LPL_B:330039,200,1008863,100,302191,100,2023-07-17T12:16:05.231Z,9.876799e+16,110428848767875249,9.956640e+16


In [74]:
new_df = []
LOOKBACK = 10
team_stats = {}


def avg_for(column, name):
    if row[column] == 100:
        team_stats[row["blue_teamid"]][name].append(1)
        team_stats[row["red_teamid"]][name].append(0)
    else:
        team_stats[row["blue_teamid"]][name].append(0)
        team_stats[row["red_teamid"]][name].append(1)

    if len(team_stats[row["blue_teamid"]][name]) > LOOKBACK:
        team_stats[row["blue_teamid"]][name].pop(0)
    if len(team_stats[row["red_teamid"]][name]) > LOOKBACK:
        team_stats[row["red_teamid"]][name].pop(0)

for _, row in df.iterrows():
    if row["blue_teamid"] not in team_stats:
        team_stats[row["blue_teamid"]] = {
            "first_inhib": [0],
            "first_tower": [0],
            "first_kill": [0],
        }
    if row["red_teamid"] not in team_stats:
        team_stats[row["red_teamid"]] = {
            "first_inhib": [0],
            "first_tower": [0],
            "first_kill": [0],
        }

    
    new_df.append({
        "platformgameid": row["platformgameid"],
        "blue_teamid": row["blue_teamid"],
        "red_teamid": row["red_teamid"],
        "blue_avg_inhib": sum(team_stats[row["blue_teamid"]]["first_inhib"]) / len(team_stats[row["blue_teamid"]]["first_inhib"]),
        "red_avg_inhib": sum(team_stats[row["red_teamid"]]["first_inhib"]) / len(team_stats[row["red_teamid"]]["first_inhib"]),
        "blue_avg_tower": sum(team_stats[row["blue_teamid"]]["first_tower"]) / len(team_stats[row["blue_teamid"]]["first_tower"]),
        "red_avg_tower": sum(team_stats[row["red_teamid"]]["first_tower"]) / len(team_stats[row["red_teamid"]]["first_tower"]),
        "blue_avg_kill": sum(team_stats[row["blue_teamid"]]["first_kill"]) / len(team_stats[row["blue_teamid"]]["first_kill"]),
        "red_avg_kill": sum(team_stats[row["red_teamid"]]["first_kill"]) / len(team_stats[row["red_teamid"]]["first_kill"]),
    })

    avg_for("killerteamid_inhib", "first_inhib")
    avg_for("killerteamid_tower", "first_tower")
    avg_for("killerteamid_kill", "first_kill")



new_df = pd.DataFrame(new_df)

wins = pd.read_csv("sql/wins.csv")
new_df = new_df.merge(wins, on="platformgameid")
display(new_df)


,platformgameid,blue_teamid,red_teamid,blue_avg_inhib,red_avg_inhib,blue_avg_tower,red_avg_tower,blue_avg_kill,red_avg_kill,winningteam
0,ESPORTSTMNT01:1110148,1.007832e+17,9.956641e+16,0.0,0.0,0.000000,0.0,0.000000,0.0,100
1,ESPORTSTMNT01:1110162,9.876799e+16,9.956641e+16,0.0,0.0,0.000000,0.0,0.000000,0.0,100
2,ESPORTSTMNT01:1110177,1.007832e+17,9.956641e+16,0.5,0.0,0.500000,0.0,0.500000,0.0,100
3,ESPORTSTMNT01:1110187,9.876799e+16,9.956641e+16,0.5,0.0,0.500000,0.0,0.500000,0.0,100
4,ESPORTSTMNT01:1110198,9.956641e+16,9.956641e+16,0.0,0.0,0.333333,0.0,0.333333,0.0,100
...,...,...,...,...,...,...,...,...,...,...
25185,LPL_B:330027,9.876799e+16,9.956640e+16,0.3,0.5,0.200000,0.5,0.500000,0.5,100
25186,LPL_B:330032,9.956640e+16,9.876799e+16,0.4,0.4,0.400000,0.3,0.500000,0.6,200
25187,LPL_B:330033,9.956640e+16,9.876799e+16,0.6,0.5,0.700000,0.4,0.400000,0.4,100
25188,LPL_B:330039,9.876799e+16,9.956640e+16,0.6,0.6,0.500000,0.6,0.300000,0.5,200


In [75]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB


X = new_df[["blue_avg_inhib",	"red_avg_inhib",	"blue_avg_tower",	"red_avg_tower",	"blue_avg_kill",	"red_avg_kill",]]
y = new_df["winningteam"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

gnb = GaussianNB()
model = gnb.fit(X_train, y_train)
predictive_labels = gnb.predict(X_test)
# print(predictive_labels)
print(accuracy_score(y_test, predictive_labels))

0.5793965859468043


In [76]:

rolling_kd = pd.read_csv("sql/kd_rolling_avg.csv")

blue_side = rolling_kd[rolling_kd["side"] == 100]
blue_side.rename(columns={
    "teamid": "blue_teamid",
    "avg_kills": "blue_avg_kills",
    "avg_deaths": "blue_avg_deaths",
    "avg_win": "blue_avg_win",
}, inplace=True)
blue_side.drop(columns=["side"], inplace=True)


red_side = rolling_kd[rolling_kd["side"] == 200]
red_side.rename(columns={
    "teamid": "red_teamid",
    "avg_kills": "red_avg_kills",
    "avg_deaths": "red_avg_deaths",
    "avg_win": "red_avg_win",
}, inplace=True)
red_side.drop(columns=[
    "side",

], inplace=True)


rolling_kd = blue_side.merge(red_side, on=["platformgameid"])
wins = pd.read_csv("sql/wins.csv")
rolling_kd = rolling_kd.merge(wins, on="platformgameid")
display(rolling_kd)


C:\Users\Jan\AppData\Local\Temp\ipykernel_33200\4041507055.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  blue_side.rename(columns={
C:\Users\Jan\AppData\Local\Temp\ipykernel_33200\4041507055.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  blue_side.drop(columns=["side"], inplace=True)
C:\Users\Jan\AppData\Local\Temp\ipykernel_33200\4041507055.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  red_side.rename(columns={
C:\U

,platformgameid,blue_teamid,blue_avg_kills,blue_avg_deaths,blue_avg_win,red_teamid,red_avg_kills,red_avg_deaths,red_avg_win,winningteam
0,ESPORTSTMNT01:1110148,1.007832e+17,16.000000,16.000000,0.500000,9.956641e+16,8.000000,24.000000,0.000000,100
1,ESPORTSTMNT01:1110162,9.876799e+16,15.000000,15.000000,0.500000,9.956641e+16,14.333333,16.333333,0.333333,100
2,ESPORTSTMNT01:1110177,1.007832e+17,15.166667,15.166667,0.500000,9.956641e+16,13.400000,16.800000,0.400000,100
3,ESPORTSTMNT01:1110187,9.876799e+16,18.571429,16.285714,0.571429,9.956641e+16,15.857143,16.857143,0.428571,100
4,ESPORTSTMNT01:1110198,9.956641e+16,18.142857,17.285714,0.571429,9.956641e+16,17.000000,17.857143,0.428571,100
...,...,...,...,...,...,...,...,...,...,...
25224,ESPORTSTMNT05:3241589,1.055940e+17,13.571429,16.571429,0.428571,1.055369e+17,15.000000,14.000000,0.428571,200
25225,ESPORTSTMNT02:3232313,1.094853e+17,12.142857,14.285714,0.428571,1.055500e+17,13.142857,11.714286,0.571429,200
25226,ESPORTSTMNT02:3231318,1.094853e+17,12.714286,13.142857,0.571429,1.055500e+17,14.142857,12.571429,0.571429,100
25227,ESPORTSTMNT02:3232320,1.055500e+17,14.142857,11.142857,0.571429,1.094853e+17,12.142857,13.714286,0.428571,100


In [77]:
X = rolling_kd[["blue_avg_kills",	"blue_avg_deaths",	"blue_avg_win",	"red_avg_kills",	"red_avg_deaths",	"red_avg_win",]]
y = rolling_kd["winningteam"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

gnb = GaussianNB()
model = gnb.fit(X_train, y_train)
predictive_labels = gnb.predict(X_test)
# print(predictive_labels)
print(accuracy_score(y_test, predictive_labels))

0.7619896948077686


In [78]:
new_df.drop(columns=["winningteam"], inplace=True)
joined_rolling = rolling_kd.merge(new_df, on=["platformgameid"])
display(joined_rolling)



,platformgameid,blue_teamid_x,blue_avg_kills,blue_avg_deaths,blue_avg_win,red_teamid_x,red_avg_kills,red_avg_deaths,red_avg_win,winningteam,blue_teamid_y,red_teamid_y,blue_avg_inhib,red_avg_inhib,blue_avg_tower,red_avg_tower,blue_avg_kill,red_avg_kill
0,ESPORTSTMNT01:1110148,1.007832e+17,16.000000,16.000000,0.500000,9.956641e+16,8.000000,24.000000,0.000000,100,1.007832e+17,9.956641e+16,0.0,0.0,0.000000,0.0,0.000000,0.0
1,ESPORTSTMNT01:1110162,9.876799e+16,15.000000,15.000000,0.500000,9.956641e+16,14.333333,16.333333,0.333333,100,9.876799e+16,9.956641e+16,0.0,0.0,0.000000,0.0,0.000000,0.0
2,ESPORTSTMNT01:1110177,1.007832e+17,15.166667,15.166667,0.500000,9.956641e+16,13.400000,16.800000,0.400000,100,1.007832e+17,9.956641e+16,0.5,0.0,0.500000,0.0,0.500000,0.0
3,ESPORTSTMNT01:1110187,9.876799e+16,18.571429,16.285714,0.571429,9.956641e+16,15.857143,16.857143,0.428571,100,9.876799e+16,9.956641e+16,0.5,0.0,0.500000,0.0,0.500000,0.0
4,ESPORTSTMNT01:1110198,9.956641e+16,18.142857,17.285714,0.571429,9.956641e+16,17.000000,17.857143,0.428571,100,9.956641e+16,9.956641e+16,0.0,0.0,0.333333,0.0,0.333333,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25195,ESPORTSTMNT05:3241589,1.055940e+17,13.571429,16.571429,0.428571,1.055369e+17,15.000000,14.000000,0.428571,200,1.055940e+17,1.055369e+17,0.6,0.6,0.600000,0.6,0.300000,0.5
25196,ESPORTSTMNT02:3232313,1.094853e+17,12.142857,14.285714,0.428571,1.055500e+17,13.142857,11.714286,0.571429,200,1.094853e+17,1.055500e+17,0.5,0.8,0.600000,0.6,0.600000,0.5
25197,ESPORTSTMNT02:3231318,1.094853e+17,12.714286,13.142857,0.571429,1.055500e+17,14.142857,12.571429,0.571429,100,1.094853e+17,1.055500e+17,0.5,0.8,0.500000,0.6,0.600000,0.6
25198,ESPORTSTMNT02:3232320,1.055500e+17,14.142857,11.142857,0.571429,1.094853e+17,12.142857,13.714286,0.428571,100,1.055500e+17,1.094853e+17,0.8,0.5,0.600000,0.6,0.500000,0.6


In [89]:
X = joined_rolling[["blue_avg_inhib",	"red_avg_inhib",	"blue_avg_tower",	"red_avg_tower",	"blue_avg_kill",	"red_avg_kill", "blue_avg_kills",	"blue_avg_deaths",	"blue_avg_win",	"red_avg_kills",	"red_avg_deaths",	"red_avg_win",]]
y = joined_rolling["winningteam"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)

gnb = GaussianNB()
model = gnb.fit(X_train, y_train)
predictive_labels = gnb.predict(X_test)
# print(predictive_labels)
print(accuracy_score(y_test, predictive_labels))

0.7930555555555555
